In [1]:
from datasets import load_dataset
import polars as pl
import os
# from dotenv import load_dotenv
# load_dotenv()


/Users/tommasofurlanello/.pyenv/versions/3.12.1/envs/modal/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Get the directory above the current directory
above_dir = os.path.dirname(os.getcwd())

# Specify the cache directory as 'data' subdirectory within the current directory
cache_dir = os.path.join(above_dir, "cache")
print(cache_dir)


/Users/tommasofurlanello/Documents/Dev/Cynde/cache


In [3]:
#use os to check if  cache_dir exists
print(os.path.exists(cache_dir))

#get the os environ variable HF_DATASETS_CACHE


True


In [4]:
dataset = load_dataset("teknium/OpenHermes-2.5", cache_dir=cache_dir)

Generating train split: 1001551 examples [00:10, 91601.74 examples/s]


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['system_prompt', 'topic', 'model', 'category', 'views', 'conversations', 'language', 'model_name', 'custom_instruction', 'idx', 'skip_prompt_formatting', 'title', 'avatarUrl', 'id', 'hash', 'source'],
        num_rows: 1001551
    })
})

In [6]:
dataset_name = "OpenHermes-2.5"
dataset_path = os.path.join(cache_dir, dataset_name)


In [7]:
dataset["train"].to_parquet(dataset_path+".parquet")

Creating parquet from Arrow format: 100%|██████████| 1002/1002 [00:03<00:00, 284.48ba/s]


1683813113

In [8]:
df = pl.read_parquet(dataset_path+".parquet")


In [9]:
df["source"].value_counts().sort("count")

source,count
str,u32
"""Econ_domain_ex…",660
"""lmsys1m""",1631
"""caseus_custom""",2688
"""LMSys Chatbot …",3136
null,4192
"""CogStackMed""",4443
"""UnnaturalInstr…",8610
"""GPT-4 Comparis…",14928
"""platypus""",22280


In [10]:
for i in df["source"].unique():
    print(i)

Econ_domain_expert
CamelAI
caseus_custom

EvolInstruct_70k
lmsys1m
cot_alpaca_gpt4
GPT-4 Comparison Data
glaive-code-assist
platypus
None
airoboros2.2
UnnaturalInstructions
LMSys Chatbot Arena
metamath
CogStackMed


In [11]:
targets = ["LMSys Chatbot Arena","caseus_custom"]
filtered_df = df.filter(pl.col("source").is_in(targets)).select("conversations","source")
filtered_path = dataset_path+"_filtered.parquet"
filtered_df.write_parquet(filtered_path)